# Exercício
## Ensembles e K-nearest neighbors
### Alunos (Nome e número usp):
 - Lucas Henrique Mantovani Jacintho - 10258942
 - Victor Luiz Fortes Rivelo - 9762960
 - Vinicius Henrique Borges - 9771546
---

Para esse exercício, vamos utilizar novamente o dataset [Breast Cancer Winsconsin](https://scikit-learn.org/stable/datasets/index.html#breast-cancer-dataset).



---

### Questão 1.

Carregue o dataset Breast Cancer do módulo `sklearn.datasets` e normalize os dados.

In [0]:
from sklearn.datasets import load_breast_cancer

X, y = load_breast_cancer(return_X_y=True)

In [0]:
from sklearn.preprocessing import scale

X = scale(X)



---

### Questão 2.

Aqui definiremos funções que serão utilizadas para simplificar as operações que faremos posteriormente. Dessa forma, implemente as funções abaixo:

*   A função `get_mean_accuracy(model, X, y)` recebe um modelo `model` e um conjunto de atributos `X` e labels `y`. Ela deve calcular a acurácia do modelo utilizando 10-fold cross-validation estratificado e retornar a acurácia média dos 10 folds
*   A função `evaluate_models(models, X, y)` recebe um conjunto de modelos definidos por um dicionário e exibe, para cada modelo, seu nome seguido da sua acurácia (calculada com a função `get_mean_accuracy`). Um exemplo de saída para o dicionário `exemplo` abaixo seria:
> A acurácia do modelo "Knn (n_neighbors = 5)" é 85.00%
>
> A acurácia do modelo "DT (gini)" é 80.00%

*   Finalmente, implemente a função `create_name_list(models)`. Essa função deve receber um dicionário e retornar uma lista contendo, para cada elemento do dicionário, uma tupla (chave, valor). Para o dicionário `exemplo`, essa função retornaria:
> `[ ('Knn (n_neighbors = 5)', KNeighborsClassifier(n_neighbors=5)), ('DT (gini)', DecisionTreeClassifier(criterion="gini"))]`


```
exemplo = {
    'Knn (n_neighbors = 5)': KNeighborsClassifier(n_neighbors=5),
    'DT (gini)': DecisionTreeClassifier(criterion="gini"),
}
```



In [0]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

def get_mean_accuracy(model, X, y):
  accuracies = []
  skf = StratifiedKFold(n_splits=10)

  for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)

    accuracies.append(score)

  return np.mean(accuracies)

def evaluate_models(models, X, y):
  for experiment, model in models.items():
    print('A acurácia do modelo "{}" é {:.2f}%'.format(experiment, get_mean_accuracy(model, X, y) * 100))

def create_name_list(models):
  return list(models.items())



---

### Questão 3.

Defina modelos de SVM (`sklearn.svm.SVC`) e MLP (`sklearn.neural_network.MLPClassifier`) para servir de *baseline* de comparação para modelos futuros. Para isso, teste ao menos 3 configurações de SVM e 3 configurações de MLP. Sua baseline será a melhor acurácia (utilizando 10-fold cross-validation estratificado) entre essas configurações. Utilize as funções definidas na Questão 2 quando necessário.

In [0]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [0]:
mlps = {
    "MLP camada escondida (5, 5)": MLPClassifier(hidden_layer_sizes=(5, 5), max_iter=5000, random_state=42),
    "MLP camada escondida (100,)": MLPClassifier(max_iter=5000, random_state=42),
    "MLP camada escondida (200,)": MLPClassifier(hidden_layer_sizes=(200), max_iter=5000, random_state=42),
}

svms = {
    "SVM poly(3)": SVC(kernel='poly', degree=3, random_state=42),
    "SVM linear": SVC(kernel='linear', random_state=42),
    "SVM rbf": SVC(random_state=42)
}

In [0]:
print("MLPs")
evaluate_models(mlps, X, y)
print()
print("SVMs")
evaluate_models(svms, X, y)

MLPs
A acurácia do modelo "MLP camada escondida (5, 5)" é 97.89%
A acurácia do modelo "MLP camada escondida (100,)" é 97.37%
A acurácia do modelo "MLP camada escondida (200,)" é 97.89%

SVMs
A acurácia do modelo "SVM poly(3)" é 90.34%
A acurácia do modelo "SVM linear" é 97.54%
A acurácia do modelo "SVM rbf" é 97.71%




---

### Questão 4.

Agora defina dois dicionários de classificadores. Um dicionário deve conter apenas classificadores KNN (`sklearn.neighbors.KNeighborsClassifier`), enquanto o outro deve conter apenas classificadores DT (`sklearn.tree.DecisionTreeClassifier`). Crie ao menos 3 configurações diferentes para cada tipo de classificador. Depois calcule e exiba a acurácia de cada configuração criada utilizando 10-fold cross-validation estratificado.

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [0]:
knns = {
  "KNN (K=5)": KNeighborsClassifier(),
  "KNN (K=2)": KNeighborsClassifier(n_neighbors=2),
  "KNN (K=10)": KNeighborsClassifier(n_neighbors=10),
}

dts = {
  "Decision Tree": DecisionTreeClassifier(random_state=42),
  "Decision Tree w/ random split": DecisionTreeClassifier(splitter='random', random_state=42),
  "Decision Tree w/ max_depth = 3": DecisionTreeClassifier(max_depth=3, random_state=42),
}

In [0]:
print("KNNs")
evaluate_models(knns, X, y)
print()
print("DTs")
evaluate_models(dts, X, y)

KNNs
A acurácia do modelo "KNN (K=5)" é 96.66%
A acurácia do modelo "KNN (K=2)" é 94.55%
A acurácia do modelo "KNN (K=10)" é 97.01%

DTs
A acurácia do modelo "Decision Tree" é 92.80%
A acurácia do modelo "Decision Tree w/ random split" é 92.97%
A acurácia do modelo "Decision Tree w/ max_depth = 3" é 90.70%




---

### Questão 5.

Agora você deve definir um dicionário contendo diferentes configurações de ensembles a serem testados. Utilize as classes `StackingClassifier`, `AdaBoostClassifier` e `VotingClassifier` do módulo `sklearn.ensemble` (fique a vontade para escolher outros tipos de ensembles desse mesmo módulo). Crie pelo menos 5 configurações diferentes tomando como estimadores os modelos definidos na Questão 4. Calcule a acurácia de cada configuração de ensemble. 

Obs.: Lembre-se de utilizar as funções definidas anteriormente

In [0]:
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier, StackingClassifier, VotingClassifier

In [0]:
print("KNNs")
estimators = create_name_list(knns)
ensembles = {
  "Stacking": StackingClassifier(estimators),
  "Voting": VotingClassifier(estimators)
}
evaluate_models(ensembles, X, y)

print()

print("DTs")
estimators = create_name_list(dts)
for name, estimator in estimators:
  ensembles = {
      "Ada Boost {}".format(name): AdaBoostClassifier(estimator),
      "Bagging {}".format(name): BaggingClassifier(estimator),
  }
  evaluate_models(ensembles, X, y)

ensembles = {
  "Extra Trees": ExtraTreesClassifier(),
  "Gradient Boosting": GradientBoostingClassifier(),
  "Random Forest": RandomForestClassifier(),
  "Stacking": StackingClassifier(estimators),
  "Voting": VotingClassifier(estimators)
}
evaluate_models(ensembles, X, y)

KNNs
A acurácia do modelo "Stacking" é 96.66%
A acurácia do modelo "Voting" é 96.84%

DTs
A acurácia do modelo "Ada Boost Decision Tree" é 91.05%
A acurácia do modelo "Bagging Decision Tree" é 94.20%
A acurácia do modelo "Ada Boost Decision Tree w/ random split" é 93.67%
A acurácia do modelo "Bagging Decision Tree w/ random split" é 96.13%
A acurácia do modelo "Ada Boost Decision Tree w/ max_depth = 3" é 96.83%
A acurácia do modelo "Bagging Decision Tree w/ max_depth = 3" é 93.86%
A acurácia do modelo "Extra Trees" é 96.67%
A acurácia do modelo "Gradient Boosting" é 96.14%
A acurácia do modelo "Random Forest" é 96.13%
A acurácia do modelo "Stacking" é 93.51%
A acurácia do modelo "Voting" é 92.98%



---

### Questão 6. 

Discuta as acurácias obtidas nas Questões 3, 4 e 5. Houve algum ganho ao utilizar Ensembles? Em qual situação houve maior ganho?

Os exemplos utilizados serão disponilizados para a resposta serão disponibilizados a seguir: 

# Resultados Questão 3

MLPs
* A acurácia do modelo "MLP camada escondida (5, 5)" é 97.89%
* A acurácia do modelo "MLP camada escondida (100,)" é 97.37%
* A acurácia do modelo "MLP camada escondida (200,)" é 97.89%

SVMs
* A acurácia do modelo "SVM poly(3)" é 90.34%
* A acurácia do modelo "SVM linear" é 97.54%
* A acurácia do modelo "SVM rbf" é 97.71%

# Resultados Questão 4

KNNs
* A acurácia do modelo "KNN (K=5)" é 96.66%
* A acurácia do modelo "KNN (K=2)" é 94.55%
* A acurácia do modelo "KNN (K=10)" é 97.01%

DTs
* A acurácia do modelo "Decision Tree" é 92.80%
* A acurácia do modelo "Decision Tree w/ random split" é 92.97%
* A acurácia do modelo "Decision Tree w/ max_depth = 3" é 90.70%

# Resultados Questão 5

KNNs
* A acurácia do modelo "Stacking" é 96.66%
* A acurácia do modelo "Voting" é 96.84%

DTs
* A acurácia do modelo "Ada Boost Decision Tree" é 91.05%
* A acurácia do modelo "Bagging Decision Tree" é 94.20%
* A acurácia do modelo "Ada Boost Decision Tree w/ random split" é 93.67%
* A acurácia do modelo "Bagging Decision Tree w/ random split" é 96.13%
* A acurácia do modelo "Ada Boost Decision Tree w/ max_depth = 3" é 96.83%
* A acurácia do modelo "Bagging Decision Tree w/ max_depth = 3" é 93.86%
* A acurácia do modelo "Extra Trees" é 96.67%
* A acurácia do modelo "Gradient Boosting" é 96.14%
* A acurácia do modelo "Random Forest" é 96.13%
* A acurácia do modelo "Stacking" é 93.51%
* A acurácia do modelo "Voting" é 92.98%


# Discussão

Ao compararmos as acurácias dos algoritmos de forma independente, e ao utilizarmos o ensemble pudemos notar que o ensemble traz ganhos, dado a comparação das as acuracias dos algotitmos das células acima, como podemos identificar ao compararmos as DT com acurácia de 92% e Ada Boost Decision Tree w/ max_depth com 96%, mas nem todos os casos apresentam melhoras, como o exemplo do Ada Boost Decision Tree onde a acurária cai para 91.05%, é interessante notar que o desempenho do MLP no melhor caso com 97.89% ainda supera a utilização de ensemble nos casos de teste utilizados, isso demonstra que o a escolha dos algoritmo que serão utilizados como estimadores requerem domínio do problema, e da execução dos mesmos, pois como podemos ver neste exemplo (que não dita a regra dos demais, mas levanta a hipótese em discussão) a estratégia que deveria trazer os melhores resultados não o fez, em grupo pensamos em duas teorias que podem responder bem este problema:
 
Uma delas é a citada sobre o conhecimento dos algoritmos e das combinações realizadas e a outra se refere ao tamanho e complexidade do problema abordado, dado que os algoritmos sem boost já apresentam resultados muito bons acima de 90% de acurácia, sendo assim é dificil obter resultados melhores sem obter um pouco de overfitting sobre o conjunto.

Vale ressaltar que utilizamos o random state = 42 para os exemplos, o motivo para tal decisão é garantir a replicabilidade do experimento, pois temos consciência de que o mesmo será corrigido e poderia gerar resultados diferentes. Em contra partida, podemos estar analisando um caso específico onde um dos classificadores tem acurária melhor, por isso levantamos a hipótese anteriormente e não concluímos em dizer que o ensemble não gera ganhos em sua aplicação.

